*Below is a complete “manual” (or методичка) that walks you through converting a command‐line image processing script into a simple Windows GUI application using Python’s built‑in **Tkinter** library.*

## 1. Survey of Python GUI Libraries

For many projects you have several choices when it comes to building a GUI in Python. Here’s a brief overview:

- **Tkinter**  
  - **Pros:**  
    - It comes with Python (no extra installation required).  
    - It is simple and well‑documented, making it perfect for school projects.  
    - It is sufficient for creating standard dialog boxes, buttons, text entries, etc.
  - **Cons:**  
    - It is not as “modern” in look as some other frameworks.

- **Pygame**  
  - **Pros:**  
    - Great for game development and interactive graphics.
  - **Cons:**  
    - It is not designed for conventional GUI applications (dialogs, forms, etc.).  
    - Overkill for a simple file‐processing tool.

- **PyQt / PySide**  
  - **Pros:**  
    - Feature‑rich and supports modern UI designs.
  - **Cons:**  
    - Has a steeper learning curve and extra installation steps, which may be more than needed for a school project.

- **Kivy**  
  - **Pros:**  
    - Excellent for multi‑touch and cross‑platform mobile/desktop apps.
  - **Cons:**  
    - More advanced and less straightforward for simple desktop apps.

**Conclusion:** For a school project where simplicity and ease of deployment are key, **Tkinter** is the best choice.

## 2. Why Choose Tkinter?

We choose Tkinter because:
- **It’s built-in:** No need for extra dependencies.
- **It is simple:** Ideal for learning the basics of event‑driven programming.
- **It meets our needs:** We can easily add text boxes, buttons, and file dialogs—all we need to wrap our command‑line script in a friendly interface.

In [ ]:
import tkinter as tk

root = tk.Tk()  # Creates the main window (the “root” widget)
root.title("My Tkinter App")  # Sets the window title
root.mainloop()  # Enters the Tkinter event loop (more on this below)

## 3. Step‑by‑Step Introduction to Tkinter

### 3.1. Creating a Basic Window

The starting point of any Tkinter application is the **main window**. This is done with:

- **`Tk()` instance:** The container for all widgets.  
- **`mainloop()`:** This method starts the event‑driven runtime cycle which waits for events (like button clicks) and updates the interface accordingly.

In [ ]:
text_entry = tk.Entry(root, width=40)
text_entry.pack(pady=5)

### 3.2. Creating an Input Box for Text

To let the user type in text (for instance, to provide a parameter or a comment), we use the **Entry** widget:

- **`Entry`:** A single‑line text input widget.  
- **`pack()`:** A simple geometry manager that adds the widget to the window.  
- **`pady=5`:** Adds vertical padding.

In [ ]:
def on_button_click():
    user_text = text_entry.get()  # Retrieve the text from the input box
    print("User entered:", user_text)  # Here, you can call your processing logic

action_button = tk.Button(root, text="Submit", command=on_button_click)
action_button.pack(pady=5)

### 3.3. Binding GUI to Python Logic

To process user input, we can add a **Button** that, when clicked, calls a function:

- **`Button`:** A clickable widget.  
- **`command`:** Specifies the function to run when the button is clicked.

In [ ]:
from tkinter import filedialog

def choose_input_file():
    file_path = filedialog.askopenfilename(
        title="Select Input Image",
        filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")]
    )
    print("Input file chosen:", file_path)
    # You can store file_path in a global variable or update a label widget

### 3.4. Letting the User Choose a File (Input)

Tkinter provides file dialog modules to select files:

- **`askopenfilename`:** Pops up a dialog for selecting an existing file.

In [ ]:
def choose_output_file():
    file_path = filedialog.asksaveasfilename(
        title="Save Output Image",
        defaultextension=".png",
        filetypes=[("PNG Image", "*.png"), ("JPEG Image", "*.jpg;*.jpeg")]
    )
    print("Output file chosen:", file_path)

### 3.5. Letting the User Choose an Output File

Similarly, to select where to save the output image, use:

- **`asksaveasfilename`:** Pops up a “save as” dialog to let the user choose a destination and filename.

### 3.6. Understanding Tkinter’s Runtime

- **The Event Loop (`mainloop`):**  
  When you call `root.mainloop()`, Tkinter enters an infinite loop where it listens for events (like button clicks or key presses) and dispatches them to your callback functions. This loop is single‑threaded by default.

- **Widgets and Instances:**  
  Each widget (like `Button`, `Entry`, or `Label`) is an instance of a Tkinter class. They are added to the main window and arranged with geometry managers (like `pack`, `grid`, or `place`).

- **Blocking Operations:**  
  Because the main loop is single‑threaded, any long‑running task executed directly (for example, processing a high‑resolution image that takes several minutes) will block the loop. This means the GUI will “freeze” until the task completes.

In [ ]:
import threading

def start_processing():
    # Validate input and output selection here…
    threading.Thread(target=process_image).start()  # Run process_image in a new thread

### 3.7. Multi‑Threading and a Loading Screen

**Why Freezing Occurs:**  
If a heavy computation (like enhancing a high‑resolution image) is performed in the main thread, the event loop cannot update the GUI until the computation finishes. This results in a frozen (non‑responsive) interface.

**Solution: Multi‑Threading**  
By running the heavy task in a separate thread, the GUI remains responsive. A simple loading screen (or status message) can notify the user that processing is in progress.

For example:

- **`threading.Thread`:** Creates a new thread to run the function `process_image` without blocking the main loop.

**Loading Screen / Status Label:**  
Update a label widget (from the main thread) to show the status of the process (e.g., “Processing…”). Note that while it is best practice to update GUI elements from the main thread, simple examples may update a status label from within the worker thread if care is taken.

### 3.8. Using Pure Functions vs. Classes

While object‑oriented design (using classes) is often recommended for large projects, for this school project we will use **pure functions**. This makes the code simpler and easier to follow:
- Each task (e.g., choosing a file, processing the image) is handled by its own function.
- Global variables (or variables stored at the module level) may be used to share state (like file paths).

In [ ]:
import tkinter as tk
from tkinter import filedialog, messagebox
import threading
import torch
import numpy as np
from PIL import Image
from basicsr.archs.rrdbnet_arch import RRDBNet  # Model architecture
from realesrgan import RealESRGANer  # The upscaling framework

# Global variables to store file paths
input_file = None
output_file = None

def choose_input_file():
    """Open a dialog to choose an input image file."""
    global input_file
    input_file = filedialog.askopenfilename(
        title="Select Input Image",
        filetypes=[("Image Files", "*.png;*.jpg;*.jpeg")]
    )
    if input_file:
        input_label.config(text=f"Input File: {input_file}")

def choose_output_file():
    """Open a dialog to choose where to save the output image."""
    global output_file
    output_file = filedialog.asksaveasfilename(
        title="Save Output Image",
        defaultextension=".png",
        filetypes=[("PNG Image", "*.png"), ("JPEG Image", "*.jpg;*.jpeg")]
    )
    if output_file:
        output_label.config(text=f"Output File: {output_file}")

def process_image():
    """
    Process the image using the upscaling model.
    This function runs in a separate thread to avoid freezing the GUI.
    """
    # Update the status label (use the main thread to update GUI if needed)
    status_label.config(text="Status: Processing...")
    try:
        # --- Original Script Logic Starts Here ---
        model_path = 'RealESRGAN_x4plus.pth'

        # Load the model weights
        state_dict = torch.load(model_path, map_location=torch.device('cpu'))['params_ema']

        # Create and load the model
        model = RRDBNet(
            num_in_ch=3, 
            num_out_ch=3, 
            num_feat=64, 
            num_block=23, 
            num_grow_ch=32, 
            scale=4
        )
        model.load_state_dict(state_dict, strict=True)

        # Initialize the upsampler
        upsampler = RealESRGANer(
            scale=4,
            model_path=model_path,
            model=model,
            tile=0,
            pre_pad=0,
            half=False
        )

        # Open the input image and convert it to an array
        input_img = Image.open(input_file).convert('RGB')
        img = np.array(input_img)

        # Enhance (upscale) the image
        output, _ = upsampler.enhance(img, outscale=4)

        # Save the output image
        output_img = Image.fromarray(output)
        output_img.save(output_file)
        # --- Original Script Logic Ends Here ---

        status_label.config(text="Status: Processing Completed!")
    except Exception as e:
        status_label.config(text=f"Status: Error - {e}")

def start_processing():
    """Validate file selections and start image processing in a new thread."""
    if not input_file or not output_file:
        messagebox.showwarning("Warning", "Please choose both input and output files.")
        return
    # Start the processing in a separate thread to keep the GUI responsive.
    threading.Thread(target=process_image, daemon=True).start()

# --- GUI Construction ---

# Create the main window (root widget)
root = tk.Tk()
root.title("Image Upscaler")

# Create a button to select the input file
choose_input_button = tk.Button(root, text="Choose Input Image", command=choose_input_file)
choose_input_button.pack(pady=5)

# Label to display the selected input file path
input_label = tk.Label(root, text="Input File: Not selected")
input_label.pack()

# Create a button to select the output file
choose_output_button = tk.Button(root, text="Choose Output Image", command=choose_output_file)
choose_output_button.pack(pady=5)


# Label to display the selected output file path
output_label = tk.Label(root, text="Output File: Not selected")
output_label.pack()

# (Optional) Create a text input box for additional parameters
text_entry = tk.Entry(root, width=40)
text_entry.pack(pady=5)
# You can retrieve its content later via: user_text = text_entry.get()

# Button to start the image processing
process_button = tk.Button(root, text="Start Processing", command=start_processing)
process_button.pack(pady=10)

# Label to display the current status (idle, processing, error, etc.)
status_label = tk.Label(root, text="Status: Idle")
status_label.pack(pady=5)

# Start the Tkinter event loop (this call blocks and handles all GUI events)
root.mainloop()

## 4. Final Updated Code Example

Below is the complete updated code that wraps the original image upscaling script in a Tkinter GUI. This code demonstrates:

- A main window with buttons to select input and output files.
- A text input box (which you may later use for additional parameters if needed).
- A “Start Processing” button that triggers the image upscaling in a separate thread.
- A status label to indicate when processing is underway or completed.

> **Note:** Make sure you have the required packages installed (e.g., `torch`, `numpy`, `Pillow`, `basicsr`, `realesrgan`).

## 5. Final Discussion

- **Tkinter Instances & Their Meaning:**  
  - **`Tk()`** creates the main application window.  
  - **`Button`**, **`Label`**, and **`Entry`** are widgets that let users interact with your program.  
  - **`pack()`** (and other geometry managers) arranges these widgets on the screen.

- **Event Loop and Runtime Cycle:**  
  - The call to **`mainloop()`** starts the event loop. This loop listens for user events (mouse clicks, key presses) and dispatches them to your callback functions.  
  - Since the loop is single‑threaded, any long computation done in the main thread will block the interface.

- **Why Interface Freezing Occurs:**  
  - If you perform heavy tasks (like image upscaling, which may take several minutes per image) in the main thread, the GUI cannot update until the task finishes.  
  - Using **multi‑threading** (via the `threading` module) moves the heavy computation to a background thread, keeping the interface responsive.

- **Pure Functions vs. Classes:**  
  - In this manual, we have used simple, pure functions to keep the code easy to follow.  
  - For larger projects, using classes (object‑oriented design) can make the code more modular and maintainable, but for this example, pure functions are perfectly acceptable.

- **Multi‑Threading & Loading Screen:**  
  - We introduced multi‑threading by wrapping the `process_image` function in a `threading.Thread` call.  
  - A status label informs the user that processing is underway, helping to improve the user experience while the background thread performs the heavy computation.

*prepared [@iliyasone](t.me/iliyasonen) and o3-mini-high*